# Training a Neural Network Model with FFBP
This notebook provides instructions on how to train a feedforward neural network using pdpyflow's FFBP package and Tensorflow. The FFBP package is intended to simplify the process of constructing a [Tensorflow Graph](https://www.tensorflow.org/programmers_guide/graphs) for neural network modeling. Tensorflow graph is a computational structure that *describes* the flow of data (tensors) through various computational operations. Thus, the processes of constructing a graph and running it are separate. 

In order to train or test a neural network we need to follow three steps:
- Prepare input data ([tutorial notebook](https://github.com/alex-ten/pdpyflow/blob/master/tutorials/building_models/prepare_input.ipynb)) [&#x21F1;](#step1)
- Construct model ([tutorial notebook](https://github.com/alex-ten/pdpyflow/blob/master/tutorials/building_models/build_model.ipynb)) [&#x21F1;](#step2)
- Run model [&#x21F1;](#step3)

We begin by importing the packages required for this tutorial and creating a [Tensorflow Graph](https://www.tensorflow.org/programmers_guide/graphs). We want to make sure that we are adding network and data queue elements to this graph.

In [2]:
import tensorflow as tf
import FFBP
tf.logging.set_verbosity(tf.logging.ERROR) # Prevent unwanted logging messages by tensorflow

FFBP_GRAPH = tf.Graph()

<a id='step2'> </a>
## Prepare Input Data
Next, we need to create `FFBP.InputData` for training and testing. Refer to the corresponding [notebook tutorial](https://github.com/alex-ten/pdpyflow/blob/master/tutorials/FFBP_network/prepare_input.ipynb) to learn more on how to create an `InputData` object. In the cell below we create two `InputData` objects, `TRAIN_DATA` and `TEST_DATA` that will be used for training and testing, respectively. Note, by convention we capitalize the names of variables that are referenced accross notebook cells (e.g. `FFBP_GRAPH`, `NUM_EPOCHS`).

In [ ]:
NUM_EPOCHS = 1000

with FFBP_GRAPH.as_default():
    
    # Create data for training
    TRAIN_DATA = FFBP.InputData(
        path_to_data_file = 'auto_data_train.txt',
        num_epochs = NUM_EPOCHS,
        batch_size = 4,
        data_len = 8,
        inp_size = 8, 
        targ_size = 8,
        shuffle_seed = SHUFFLE
    )
    # Create data for testing
    TEST_DATA = FFBP.InputData(
        path_to_data_file = 'auto_data_test.txt',
        num_epochs = NUM_EPOCHS,
        batch_size = 15,
        inp_size = 8, 
        targ_size = 8,
        data_len = 15
    )

<a id='step1'> </a>
## Construct Model
Now we outline the network structure and specify the flow of data through it. A more detailed description of how this is done is provided in a separate [tutorial](https://github.com/alex-ten/pdpyflow/blob/master/tutorials/building_and_training_FFBP_network/connect_layers.ipynb). We set the same weight initialization range for the hidden and output layers, but it can be controlled individually for each layer.

In [ ]:
wr = 0.25

# Add network components to the graph
with FFBP_GRAPH.as_default():
    model_name = 'autoencoder'
    with tf.name_scope(model_name):
        
        # Create input and target placeholder
        input_  = tf.placeholder(dtype = tf.float32, shape=[None, 8], name='model_inp')
        target = tf.placeholder(dtype = tf.float32, shape=[None, 8], name='targets')
        
        # Create first hidden layer
        hidden_layer = FFBP.BasicLayer(
            layer_name = 'hidden_layer', 
            layer_input = input_, 
            size = 3, 
            wrange = [-wr, wr], 
            nonlin = tf.nn.sigmoid, 
            bias = True, 
            seed = None
        )
        
        # Create another first-level hidden layer
        output_layer = FFBP.BasicLayer(
            layer_name = 'output_layer', 
            layer_input = hidden_layer.output, 
            size = 8, 
            wrange = [-wr, wr], 
            nonlin = tf.nn.sigmoid, 
            bias = True, 
            seed = None
        )
        
        MODEL = FFBP.Model(
            name = model_name,
            layers = [hidden_layer, output_layer],
            train_data = train_data, 
            inp        = input_,
            targ       = target,
            loss       = tf.reduce_sum(tf.squared_difference(target, output_layer.output), name='loss_function'),
            optimizer  = tf.train.MomentumOptimizer(lr, m),
            test_data  = test_data
        )

<a id='step3'> </a>
## Run Model
The model is run inside two for-loops (one nested inside the other): the (inner) train loop and the (outer) run loop. In a single iteration of the run loop, model parameters will be initialized either randomly or by a restoration from an existing checkpoint directory. A single iteration of the inner train loop corresponds to a single epoch of training/testing. Thus, minimally, a valid run code would look something like:
```python
for run_ind in range(NUM_RUNS):

    with tf.Session(graph=FFBP_GRAPH) as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        coordinator = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coordinator)
        
        for i in range(NUM_EPOCHS):
            testloss, snap = MODEL.test_epoch(session=sess, verbose=True)
            loss = MODEL.train_epoch(session=sess, verbose=False)

        coordinator.request_stop()
        coordinator.join(threads)

```
Here, we start a new Tensorflow session for each run iteration and iterate over NUM_RUNS. Within each session we initilize global and local variables (this reinitializes model parameters and local queue variables),  start the queues, and run the train loop. Inside the train loop we test and train the model for NUM_EPOCHS number of times.

In order to make the runs a bit more nuanced we need to add a few elements to this code.  like saving test data and/or saving and restoring a model from a checkpoint, we need

In [ ]:
# Set up run parameters
NUM_RUNS = 1
TEST_EPOCHS = [0,1,3,5,30,60,120,180,270,300]
SAVE_EPOCHS = [NUM_EPOCHS-1]
ECRIT = 0.01

# Create ModelSaver
saver = FFBP.ModelSaver(restore_from=None, make_new_logdir=True)

for run_ind in range(NUM_RUNS):
    print('>>> RUN {}'.format(run_ind))
    
    with tf.Session(graph=FFBP_GRAPH) as sess:

        # restore or initialize FFBP_GRAPH variables:
        start_epoch = saver.init_model(session=sess)

        # create coordinator and start queue runners
        coordinator = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coordinator)

        for i in range(start_epoch, start_epoch + NUM_EPOCHS):
            # Test model occasionally
            if any([i==test_epoch for test_epoch in TEST_EPOCHS]):
                testloss, snap = MODEL.test_epoch(session=sess, verbose=True)
                saver.save_test(snap, run_ind)

            # Run one training epoch
            loss = MODEL.train_epoch(session=sess, verbose=False)
            saver.save_loss(loss, run_ind)

            # Save model occasionally
            if any([i==save_epoch for save_epoch in SAVE_EPOCHS]):
                saver.save_model(session=sess, model=MODEL)

            # Do final test, stop queues, and break out from training loop
            if loss < ECRIT or i == start_epoch + (NUM_EPOCHS - 1): 
                print('Final test ({})'.format(
                    'loss < ecrit' if loss < ECRIT else 'num_epochs reached'))

                testloss, snap = MODEL.test_epoch(session=sess, verbose=True)
                saver.save_test(snap, run_ind)

                coordinator.request_stop()
                coordinator.join(threads)

                saver.save_model(session=sess, model=MODEL)
                break